In [16]:
import uol_redacoes_xml
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import mac_morpho
import nltk.corpus
import pandas as pd
import regex as re
import spacy

In [14]:
essays = uol_redacoes_xml.load()

Separar elementos com a mesma função sintática

Antes de conjunções adversativas **x**

Separar orações intercaladas

aposto **x**

Separar expressões explicativas

Isolar o vocativo

Isolar datas

Marcar zeugma

Separar as orações subordinadas adverbiais

Separar as orações subordinadas adverbiais deslocadas

Não separar o sujeito do predicado **x**

Não separar o verbo do complemento **x**

In [1]:
def count_virgula(palavra1,palavra2,frase):
    if palavra1 == None or palavra2  == None or palavra1 == '' or  palavra2 == '':
        return 0
    posicao_gato = frase.find(palavra1)
    posicao_pulou = frase.find(palavra2)

    if posicao_gato != -1 and posicao_pulou != -1:
        substring = frase[posicao_gato:posicao_pulou]
        numero_virgulas = substring.count(',')
        return numero_virgulas
    else:
        return -1

In [2]:
def sujeito_pedicado(frase):
    nlp = spacy.load('pt_core_news_sm')
    doc = nlp(frase)

    # Identificar sujeito e predicado
    sujeito = ""
    predicado = ""

    for token in doc:
        if "subj" in token.dep_ and sujeito == "":
            sujeito += token.text
        elif "ROOT" in token.dep_ and predicado == "":
            predicado += token.text

    return {'sujeito': sujeito, 'predicado': predicado}

In [3]:
def error_virgula_suj_pred(frase):
    senter = sujeito_pedicado(frase)

    num = count_virgula(senter['sujeito'],senter['predicado'], frase)
    if num%2 == 0:
        return 0
    elif num == -1:
        return -1
    else:
        return 1

In [4]:
def identificar_verbo_complemento(frase):
    nlp = spacy.load("pt_core_news_sm")
    doc = nlp(frase)
    
    verb = None
    obj = None
    
    for token in doc:
        if "VERB" in token.pos_:
            verb = token.text
            for child in token.children:
                if "obj" in child.dep_:
                    obj = child.text
                    break
    
    return {'verbo': verb, 'obj': obj}


In [5]:
def error_virgula_verbo(frase):
    senter = identificar_verbo_complemento(frase)
    num = count_virgula(senter['verbo'],senter['obj'], frase)
    if num%2 == 0:
        return 0
    elif num == -1:
        return -1
    else:
        return 1

In [6]:
def error_virgula_Conj_adj(frase):
    conjuncoes_adversativas = ['mas', 'porém', 'contudo', 'todavia', 'entretanto', 'no entanto']
    frase = frase.lower() 
    palavras = frase.split()

    for i in range(1, len(palavras)):
        if palavras[i] in conjuncoes_adversativas:
            if palavras[i - 1][-1] == ',':
                return 0
            else:
                return 1

    return -1

In [7]:
def error_virgulas_aposto(frase):
    nlp = spacy.load('pt_core_news_sm')
    doc = nlp(frase)

    for token in doc:
        if token.dep_ == 'appos':
            
            rela_appos = [str(child) for child in token.children]
            print(token, rela_appos)
            if rela_appos.count(',') == 2:
                return 0
            else: return 1
    else:
        return -1

In [8]:
def corretor(frase):
    error = 0
    message = ""
    
    e = error_virgula_verbo(frase)
    if e ==1:
        error += e
        message += "Comma separating verb and complement; "
    
    e = error_virgula_suj_pred(frase)
    if e == 1:
        error += e
        message += "Comma separating Subject from predicate; "
    
    e = error_virgula_Conj_adj(frase)
    if e ==1:
        error += e
        message += "Missing comma before adverbial conjunction; "
    
    e = error_virgulas_aposto(frase)
    if e ==1:
        error += e
        message += "Missing comma between the 'aposto'; "

    return error, message

In [9]:
def split_sentences(text):
    regex =  r'(?<=[.!?])\s*(?=(?:[^"]*"[^"]*")*[^"]*$)'  # Expressão regular para separar as sentenças

    sentences = re.split(regex, text)

    return sentences

In [10]:
def get_essays_corrections_tool(essays):
    data = []
    for i, ess in enumerate(essays):
        sentences = split_sentences(ess.text.replace('\n', ''))
        for sent in sentences:
            if sent != "":
                error, message = corretor(sent)
                data.append({
                        'essay': i,
                        'error': error,
                        'message': message,
                        'frase': sent})
    return pd.DataFrame(data)

In [ ]:
erros_df = get_essays_corrections_tool(essays)

In [19]:
erros_df.head(20)

,essay,error,message,frase
0,0,0,,"A liberdade de expressão, assim como o direito..."
1,0,0,,"Quando um entra em contato com o outro, é prec..."
2,0,0,,Em épocas de regimes autoritários como as dita...
3,0,0,,"Nessas épocas, as pessoas também sofriam grave..."
4,0,0,,"Porém, há casos em que essas garantias são pos..."
5,0,1,Comma separating Subject from predicate;,"Assim, o alvo dessas matérias, pode requerer s..."
6,0,0,,"Portanto, é preciso avaliar até onde o direito..."
7,0,0,,"Dessa forma, o bem estar do ser humano deve es..."
8,1,1,Missing comma between the 'aposto';,Os direitos fundamentais contidos na Constitui...
9,1,1,Missing comma between the 'aposto';,Esse é o caso da decisão judiciária aplicada a...


In [20]:
erros_df.to_csv('Comma_error.csv', encoding='utf-8')